In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import torchvision.transforms as transforms
import numpy as np
import os
import random
import numpy as np
import torch
from PIL import Image

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)


In [ ]:
from class_mapping import ID_TO_NAME, NAME_TO_ID, ID_TO_NAME_MAPPING

NUM_CLASSES = len(ID_TO_NAME)

In [ ]:
from torch.utils.data import DataLoader, Dataset, random_split

CLASSIFICATION_DATASET_PATH = "data/classification_dataset"
WEIGHTS_PATH = None #r"checkpoints\ckpt_epoch_006.pt"
LOAD_WEIGHTS = True  # set to False if you want to train from scratch
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-3
BATCH_SIZE = 32

In [ ]:

NUM_EPOCHS = 100
START_EPOCH = 0

In [4]:
def check_class_dirs(base_path: str):
    """
    For each class in ID_TO_NAME_MAPPING, verify if a directory named
    after the class exists under base_path. If not, print it.
    """
    missing = []
    for class_id, (name, _) in ID_TO_NAME_MAPPING.items():
        # Sanitize or normalize name if needed (e.g., replace spaces)
        dir_name = "class_" + name.lower().replace(" ","_")
        dir_path = os.path.join(base_path, dir_name)
        if not os.path.isdir(dir_path):
            missing.append([dir_name,class_id])

    if missing:
        print("The following class directories are missing:")
        for name in missing:
            print(f" - {name}")
    else:
        print("All class directories are present.")
    

if __name__ == "__main__":
    # Example usage
    base_folder = "classifier_data"
    check_class_dirs(base_folder)

All class directories are present.


In [ ]:
class CropWeedDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.root = []
        for folder in os.listdir(image_dir):
            folder_path = os.path.join(image_dir, folder)
            for file in os.listdir(folder_path):
                self.root.append(
                    (folder[6:].replace("_", " "), os.path.join(folder_path, file))
                )
        self.transform = transform

    def __len__(self):
        return len(self.root)

    def __getitem__(self, idx):
        label, image_path = self.root[idx]
        label_id = NAME_TO_ID[label.lower()]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label_id


In [6]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.25]*3)
])

In [ ]:
dataset = CropWeedDataset(image_dir=CLASSIFICATION_DATASET_PATH, transform=transform)  
print(len(dataset))  

sample = dataset[random.randint(3, len(dataset))]
print(sample[0].shape)
print(sample[1])

73824
torch.Size([3, 256, 256])
33


In [ ]:
#import models
from classification_models.MobileNet import MobileNetV2
from classification_models.ResNet18 import ResNet18
from classification_models.ShuffleNet import ShuffleNet
from classification_models.ShuffleNet_SE import ShuffleNetV2WithSE
from classification_models.ShuffleNet_SEPCONV import ShuffleNetV2WithSepConv
from classification_models.ShuffleNet_SEPCONV_SE import ShuffleNetV2WithSepConvAndSE
from classification_models.SqueezeNet import SqueezeNet

In [16]:
from torch.utils.tensorboard import SummaryWriter

def run_training(model, train_loader, val_loader, num_epochs=20, start=0, lr=1e-3, optimizer=None, log_dir="./runs/", save_dir="./checkpoints/"):
    os.makedirs(save_dir, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    writer = SummaryWriter(log_dir=log_dir)
    global_step = 0

    for epoch in range(start, start + num_epochs):
        model.train()
        train_loss = train_correct = train_total = 0
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{start + num_epochs} [Train]")
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_size = labels.size(0)
            train_loss += loss.item() * batch_size
            preds = outputs.argmax(dim=1)
            train_correct += preds.eq(labels).sum().item()
            train_total += batch_size

            batch_acc = 100. * preds.eq(labels).sum().item() / batch_size
            train_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{batch_acc:.2f}%")
            writer.add_scalar("Train/Batch_Loss", loss.item(), global_step)
            writer.add_scalar("Train/Batch_Acc", batch_acc, global_step)
            global_step += 1

        epoch_train_loss = train_loss / train_total
        epoch_train_acc = 100. * train_correct / train_total
        writer.add_scalar("Train/Epoch_Loss", epoch_train_loss, epoch)
        writer.add_scalar("Train/Epoch_Acc", epoch_train_acc, epoch)

        # Validation
        model.eval()
        val_loss = val_correct = val_total = 0
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch}/{start + num_epochs} [Val]")
        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                batch_size = labels.size(0)
                val_loss += loss.item() * batch_size
                preds = outputs.argmax(dim=1)
                val_correct += preds.eq(labels).sum().item()
                val_total += batch_size

                batch_acc = 100. * preds.eq(labels).sum().item() / batch_size
                val_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{batch_acc:.2f}%")

        epoch_val_loss = val_loss / val_total
        epoch_val_acc = 100. * val_correct / val_total
        writer.add_scalar("Val/Epoch_Loss", epoch_val_loss, epoch)
        writer.add_scalar("Val/Epoch_Acc", epoch_val_acc, epoch)

        print(f"Epoch {epoch:2d} | Train Loss: {epoch_train_loss:.4f}, Acc: {epoch_train_acc:.2f}% | Val Loss: {epoch_val_loss:.4f}, Acc: {epoch_val_acc:.2f}%")

        # Checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': epoch_train_loss,
            'val_loss': epoch_val_loss,
        }, os.path.join(save_dir, f"ckpt_epoch_{epoch:03d}.pt"))

        scheduler.step(epoch_val_loss)

    writer.close()

In [ ]:
if __name__ == "__main__":
    check_class_dirs(CLASSIFICATION_DATASET_PATH)
    
    dataset = CropWeedDataset(image_dir=CLASSIFICATION_DATASET_PATH, transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    model = ShuffleNetV2WithSE(num_classes=NUM_CLASSES).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    #load pretrained weights if available
    if LOAD_WEIGHTS:
        state_dict = torch.load(WEIGHTS_PATH, map_location=device)  # or 'cuda' if using GPU
        model.load_state_dict(state_dict["model_state_dict"])
        optimizer.load_state_dict(state_dict['optimizer_state_dict'])
        start = state_dict.get('epoch', -1) + 1
        print(f"✅ Loaded weights from {WEIGHTS_PATH}")

    run_training(model, train_loader, val_loader, num_epochs=NUM_EPOCHS, start=START_EPOCH, lr=lr, optimizer=optimizer)

All class directories are present.
✅ Loaded weights from C:\Users\PC\Documents\Eman\checkpoints\ckpt_epoch_006.pt


Epoch 7/107 [Val]: 100%|████████████████████████████████████| 462/462 [01:47<00:00,  4.28it/s, acc=76.92%, loss=0.8564]


Epoch  7 | Train Loss: 1.2007, Acc: 65.85% | Val Loss: 1.2487, Acc: 64.65%


Epoch 8/107 [Val]: 100%|████████████████████████████████████| 462/462 [01:49<00:00,  4.23it/s, acc=76.92%, loss=1.1180]


Epoch  8 | Train Loss: 1.1453, Acc: 67.32% | Val Loss: 1.2714, Acc: 63.51%


...

Epoch 42 | Train Loss: 0.5069, Acc: 84.42% | Val Loss: 0.7921, Acc: 77.97%


Epoch 43/107 [Val]: 100%|███████████████████████████████████| 462/462 [01:47<00:00,  4.29it/s, acc=76.92%, loss=0.6297]


Epoch 43 | Train Loss: 0.5079, Acc: 84.78% | Val Loss: 0.7922, Acc: 78.10%


Epoch 44/107 [Train]:   6%|█▉                              | 115/1846 [00:33<08:26,  3.42it/s, acc=81.25%, loss=0.7158]


KeyboardInterrupt: 